In [1]:
# Imports
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_openai import ChatOpenAI # Note the change from OpenAI to ChatOpenAI
import sqlite3
import datetime
import chromadb


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY_3")
collection = chromadb.PersistentClient(path="databases/").get_collection(name="questions_code_examples_python") # Took 36.5s to deal with this part (maybe downloading embedding function part)

c:\Users\voczie\Documents\UFPB\TCC\genomechat\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load database
db = sqlite3.connect('sapiens_gff.db')
cursor = db.cursor()

In [2]:
# Modify this cell each test!!!!!!!!!!!!!!!!!!!!!

test_number = 4

if os.path.exists('results/') == False:
    os.mkdir('results/')

if os.path.exists(f'results/Teste {test_number}/') == False:
    os.mkdir(f'results/Teste {test_number}/')

if os.path.exists(f'results/Teste {test_number}/no-prompt') == False:
    os.mkdir(f'results/Teste {test_number}/no-prompt')

if os.path.exists(f'results/Teste {test_number}/with-prompt') == False:
    os.mkdir(f'results/Teste {test_number}/with-prompt')

if os.path.exists(f'results/Teste {test_number}/no-prompt/gpt4omini') == False:
    os.mkdir(f'results/Teste {test_number}/no-prompt/gpt4omini')

if os.path.exists(f'results/Teste {test_number}/with-prompt/gpt4omini') == False:
    os.mkdir(f'results/Teste {test_number}/with-prompt/gpt4omini')

In [3]:
with open("1.2 - prompt.md", "r", encoding="utf-8") as f:
    context = f.read()

In [4]:
ddl = '''table_name,field_name,field_type,field_description
features,id,text,Primary key for features. The content of this field is determined by the user and the file format at creation time
features,seqid,text,Chromosome or contig ID
features,source,text,The source of the annotation (e.g. Ensembl; GENCODE)
features,featuretype,text,The type of feature (e.g. gene; transcript; exon)
features,start,int,The start coordinate of the feature
features,end,int,The end coordinate of the feature
features,score,text,A score associated with the feature (if available)
features,strand,text,The strand of the feature (+; -; or .)
features,frame,text,The coding frame of the feature (0; 1; or 2)
features,attributes,text,A string containing semicolon-separated attribute-value pairs
features,extra,text,A JSON-serialized list of non-standard extra fields. These are sometimes added by analysis tools (e.g. BEDTools). For standard GFF/GTF this field will be empty
features,bin,int,The genomic bin according to the UCSC binning strategy
features,primary key,(id),Same as the field id. Primary key for features
relations,parent,text,Foreign key to features.id – a gene for example
relations,child,text,Foreign key to feature.id – an mRNA or an exon for example
relations,level,int,In graph terms the number of edges between child and parent. In biological terms if parent=gene and child=mRNA then level=1. If parent=gene and child=exon then level=2
relations,primary key,(parent, child, level),Composite key using the fields parent; child and level. Composed of two foreign keys and one integer
meta,dialect,text,A JSON-serialized version of the dialect empirically determined when parsing the original file
meta,version,text,The gffutils version used to create the database
directives,directive,text,String directive without the leading ##
autoincrements,base,text,By default the feature type (gene; exon; etc) but can also be the value of any GFF field or attribute (e.g. the seqid or “GENE_1” (in the case of multiple features with ID=”GENE_1”)
autoincrements,n,int,Current extent of autoincrementing – add 1 to this when autoincrementing next time
autoincrements,primary key,(base),Same as the field base. Primary key for autoincrements
duplicates,idspecid,text,ID of the identified duplicated feature
duplicates,newid,text,New ID for the duplicated feature
duplicates,primary key,(newid),Same as the field newid. Primary key for duplicates
'''

In [12]:
questions = [
    """How many genes and transcripts are there on chromosome 3?""",  #1
    """How many protein-coding genes are on chromosome 12?""",        #2
    """How many lncRNA genes are on chromosome 7?""",                 #3
    """How many pseudogenes are on the X chromosome?""",              #4
    """How many genes for miRNA exist in chromosome 10?""",           #5
    """Calculate the sizes of each gene locus separately: XIST, MALAT1, BRCA1, COL1A2, NFKB1, NFKB2, REL, RELA and RELB""",       #6
    """How many transcript isoforms does the XIST gene have? Print the transcript isoform names (transcript_name) and the sizes of each.""",      #7
    """How many exons does the XIST gene have?""",        #8                                                                                        
    """How many exons does each transcript isoform of the BRCA1 gene have? Print the transcript isoform names (tr anscript_name) and the number of exons.""",         #9
    """What is the average exon size of the BRCA1 transcript isoforms?""",        #10
    """What is the chromosomal position of the BRCA1 gene?""",        #11
    """On which chromosomes are the genes NFKB1, NFKB2, REL, RELA and RELB located?""",       #12
    """Print the names of the 2 genes located immediately before and after the gene COL1A2, respectively""",      #13
    """What is the biotype of the following genes XIST, MALAT1, BRCA1,NFKB1, NFKB2, REL, RELA, RELB and COL1A2?""",        #14
    """What strand are the HOTAIR, HOXC11, and HOXC12 genes located on?""",       #15
    """Which genes are located between the HOXC11 and HOXC12 genes on + and - strands?""",        #16
    """Get the following informations about each transcript isoforms of the XIST, MALAT1, BRCA1, NFKB1, COL1A2, HOTAIR, HOXC11, and HOXC12 genes: chromosomal location and position, size, number of exons, average exon size, strand, and biotype. Organize all the information in a table and save it.""",      #17
    """Generate a scatterplot to represent the distribution of gene sizes in the X chromosome.""",        #18
    """Generate a stacked barplot chart to represent the proportions of protein-coding, lncRNA and miRNA genes on each chromosome separately.""",         #19
    """Generate a boxplot to represent the comparison of protein_coding, lncRNA, and miRNA transcript sizes"""        #20
]

In [ ]:
questions_2 = [
    """How many genes and transcripts are there on chromosome Y?""",  #1 Gene Count and Distribution by Chromosome
    """How many protein-coding transcripts are present on chromosome 1?""",        #2 Gene Count and Distribution by Chromosome
    """What are the Ensembl gene IDs and biotypes of XIST, MALAT1, BRCA1, and COL1A2?""",    #3 Specific Gene Characteristics
    """Which BRCA1 transcript has the smallest average exon size?""",              #4 Specific Gene Characteristics
    """Which gene lies directly downstream of COL1A2 on the same strand?""",           #5 Chromosomal Location
    """Which gene precedes COL1A2 and is on the same strand?""",       #6 Chromosomal Location
    """Create a table showing the number of transcripts per gene for BRCA1, MALAT1, and XIST.""",      #7 Gene and Isoform Features
    """Generate a scatterplot showing the relationship between gene length and number of exons on chromosome X."""        #8 Gene and Isoform Features                                                                                       
]

In [6]:
def get_code_examples(question_query, n = 3):
    results = collection.query(
        query_texts=[question_query],
        n_results=3,
        include=['metadatas', 'documents']
    )

    dict_results = {}
    for i, doc in enumerate(results['documents'][0]):
        dict_results.update({doc:results['metadatas'][0][i]['code_example']})

    return dict_results

In [7]:
rate_limiter = InMemoryRateLimiter(requests_per_second=2, check_every_n_seconds=0.1, max_bucket_size=10)

In [21]:
# Use ChatOpenAI instead of OpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1, rate_limiter=rate_limiter, api_key=OPENAI_API_KEY)

current_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

for ix, question in enumerate(questions_2):
    questions_codes = get_code_examples(question)

    TEMPLATE = '''
    {context}

    ### Given a genome annotation in a GFF/GTF format, and all of its data is stored in a SQLite3 database, whose fields are described within the DDL: 
    <ddl>
    {ddl}
    </ddl>

    <user_question>
    Guide me with a query or a walkthrough based on to answer the following question: {question}
    </user_question>

    Here's some code examples in Python 3 that might help:
    <code_examples>
    {questions_codes}
    </code_examples>
    '''

    prompt = PromptTemplate(
        input_variables=["context", "ddl", "question", "questions_codes"], template=TEMPLATE
    )

    chain = prompt | llm | StrOutputParser()

    # Correct way to invoke the chain with a dictionary of inputs
    result = chain.invoke({
        "context": context,
        "ddl": ddl,
        "question": question,
        "questions_codes": questions_codes
    })

    with open(f'results/Teste {test_number}/with-prompt/gpt4omini/llm_test_question_{ix}_{current_time}.txt', 'w', encoding='utf-8') as f:
        f.write(result)

In [9]:
# Use ChatOpenAI instead of OpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1, rate_limiter=rate_limiter, api_key=OPENAI_API_KEY)

current_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

for ix, question in enumerate(questions_2):
    questions_codes = get_code_examples(question)
    
    TEMPLATE = '''
    ### Given a genome annotation in a GFF/GTF format, and all of its data is stored in a SQLite3 database, whose fields are described within the DDL: 
    <ddl>
    {ddl}
    </ddl>

    <user_question>
    Guide me with a query or a walkthrough based on to answer the following question: {question}
    </user_question>

    Here's some code examples in Python 3 that might help:
    <code_examples>
    {questions_codes}
    </code_examples>
    '''

    prompt = PromptTemplate(
        input_variables=["ddl", "question", "questions_codes"], template=TEMPLATE
    )

    chain = prompt | llm | StrOutputParser()

    # Correct way to invoke the chain with a dictionary of inputs
    result = chain.invoke({
        "ddl": ddl,
        "question": question,
        "questions_codes": questions_codes
    })

    with open(f'results/Teste {test_number}/no-prompt/gpt4omini/llm_test_question_{ix}_{current_time}.txt', 'w', encoding='utf-8') as f:
        f.write(result)

In [44]:
query = '''
SELECT 'gene' AS feature_type, COUNT(*) AS count
FROM features
WHERE seqid = '3' AND featuretype = 'gene'
GROUP BY feature_type

UNION ALL

SELECT 'transcript' AS feature_type, COUNT(*) AS count
FROM features
WHERE seqid = '3' AND featuretype IN ('transcript')
GROUP BY feature_type;
'''

cursor.execute(query)
results = cursor.fetchall()
results

[('gene', 1108), ('transcript', 1660)]

In [45]:
genes_chr3 = sum(1 for feature in db.features_of_type('gene') if feature.seqid == '3')
transcripts_chr3 = sum(1 for feature in db.features_of_type('transcript') if feature.seqid == '3')
print(f"Genes on chromosome 3: {genes_chr3}")
print(f"Transcripts on chromosome 3: {transcripts_chr3}")

Genes on chromosome 3: 1108
Transcripts on chromosome 3: 1660
